In [10]:
%pip install pandas
%pip install geopandas
%pip install dill
%pip install bokeh
%pip install panel

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import geopandas as gpd

import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import GeoJSONDataSource, HoverTool, ColorBar, LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.transform import factor_cmap
from bokeh.palettes import Blues9, RdYlGn11, Viridis256


In [12]:
from data_loader import get_geo_data, get_citibike_data

car_data, nbh_data = get_geo_data()
items_df, station_df = get_citibike_data()

merged_df = pd.merge(items_df, station_df, on="station_id")

car_df = pd.DataFrame(car_data)

print(merged_df)

                                station_id  last_reported  \
0     66dc7ba2-0aca-11e7-82f6-3863bb44ef7c          86400   
1     66dc1687-0aca-11e7-82f6-3863bb44ef7c          86400   
2     498d7e8e-5164-4f26-9ce6-399947d73b1a     1759761514   
3                      1905837242740508940          86400   
4     44fa161f-eb62-459b-b53e-da83788cfa2a     1763474785   
...                                    ...            ...   
2315  cbc48689-7805-49dd-9669-5471f4b7b6fb     1765923089   
2316  519824e4-69ba-4270-a395-17c204f328f8     1765923089   
2317  bd6f422b-d7ae-4d7e-9261-653fdd8e6888     1765923052   
2318  66dddd28-0aca-11e7-82f6-3863bb44ef7c     1765923040   
2319  581211b2-4e42-48f2-8a8f-5f968cb1c5df     1765923071   

                legacy_id  num_ebikes_available  eightd_has_available_keys  \
0                     511                     0                      False   
1                     437                     0                      False   
2                    3849        

### Merging Data:

In [13]:
import numpy as np


bike_gdf = gpd.GeoDataFrame(
    merged_df,
    geometry=gpd.points_from_xy(merged_df.lon, merged_df.lat))
bike_gdf = bike_gdf.set_crs("EPSG:4326", allow_override=True)
bike_gdf = bike_gdf.to_crs(epsg=3857)
merged_gdf = gpd.sjoin(bike_gdf, car_data[['geometry', 'name']], predicate='within')
merged_gdf = merged_gdf.rename(columns={'name_right': 'zip_code'})


station_counts = merged_gdf.groupby('zip_code').size().reset_index(name='station_count')
#print(station_counts)
merged_data = car_data.merge(station_counts, left_on='name', right_on='zip_code', how='left')
merged_data = merged_data.rename(columns={'station_count_y': 'station_count'})

merged_data = merged_data.drop(columns=['zip_code'])
merged_data = merged_data.drop(columns=[col for col in car_data.columns if col.startswith('B25')])
merged_data['station_count'] = merged_data['station_count'].fillna(0)
print(merged_data)

merged_data['car_class'] = pd.qcut(merged_data['pct_vehicle'], 3, labels=[0,1,2], duplicates='drop')
merged_data['bike_class'] = pd.cut(merged_data['station_count'], bins=[-1, 0, 5, np.inf], labels=[0, 1, 2])


merged_data['bivariate_class'] = merged_data['car_class'].astype(int) * 3 + merged_data['bike_class'].astype(int)
print(merged_data)

            geoid   name  households  \
0    86000US10001  10001     15097.0   
1    86000US10002  10002     35771.0   
2    86000US10003  10003     25080.0   
3    86000US10004  10004      1775.0   
4    86000US10005  10005      5156.0   
..            ...    ...         ...   
191  86000US11691  11691     22382.0   
192  86000US11692  11692      7981.0   
193  86000US11693  11693      5405.0   
194  86000US11694  11694      8046.0   
195  86000US11697  11697      1668.0   

                                              geometry  no_vehicle  \
0    MULTIPOLYGON (((-8238766.757 4975921.508, -823...     12699.0   
1    MULTIPOLYGON (((-8237364.465 4970257.948, -823...     29144.0   
2    MULTIPOLYGON (((-8237571.742 4972792.277, -823...     20239.0   
3    MULTIPOLYGON (((-8239611.227 4968758.136, -823...      1427.0   
4    MULTIPOLYGON (((-8239034.703 4969185.603, -823...      4488.0   
..                                                 ...         ...   
191  MULTIPOLYGON (((-8214183

In [14]:
from pyproj import Transformer
from bokeh.models import ColumnDataSource
from bokeh.models import WheelZoomTool

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
merged_df['MercatorX'], merged_df['MercatorY'] = transformer.transform(merged_df['lon'].values, merged_df['lat'].values)
merged_df['icon'] = 'https://img.icons8.com/ios-glyphs/30/bicycle.png'
bike_source = ColumnDataSource(merged_df)


In [27]:
def create_census_map():
    bivariate_colors = {
    0: '#e8e8e8', 1: '#ace4e4', 2: '#5ac8c8',
    3: '#dfb0d6', 4: '#a5add3', 5: '#5698b9',
    6: '#be64ac', 7: '#8c62aa', 8: '#3b4994'
    }
    merged_data['color'] = merged_data['bivariate_class'].map(bivariate_colors)
    car_data_merc = merged_data.to_crs(epsg=3857)
    bounds = car_data_merc.total_bounds
    p = figure(
        title="Test",
        x_range=(bounds[0], bounds[2]),
        y_range=(bounds[1], bounds[3]),  
        tools='pan,zoom_in,zoom_out,reset,save,hover',
        width=1200,
        height=800,
        toolbar_location='above'
    )
    
    p.add_tile("CARTODBDARKMATTER")
    
    metric_min = car_data['pct_no_vehicle'].min()
    metric_max = car_data['pct_no_vehicle'].max()
    
    
    
    print(car_data_merc)
    geosource = GeoJSONDataSource(geojson=car_data_merc.to_json())
    
    patches = p.patches(
        'xs', 'ys',
        source=geosource,
        fill_color='color',
        fill_alpha=0.6, 
        line_color='white',
        line_width=0.5,
        hover_fill_alpha=0.7,
        hover_line_color='black'
    )
    print("test", merged_data[['bivariate_class', 'color']].head())
    #p.circle(x='MercatorX', y='MercatorY', radius=2, source=bike_source)
    #p.image_url(url='icon', x='MercatorX', y='MercatorY', w=10, h=10, 
            #anchor="center", source=bike_source, w_units='screen', h_units='screen')
    #p.image_url(url='icon', x='MercatorX', y='MercatorY', w=30, h=30, anchor="center", source=bike_source)
    hover = p.select_one(HoverTool)
    hover.tooltips = [
        ('ZIP Code', '@name'),
        ('Households', '@households'),
        ('Vehicle Ownership', '@pct_vehicle{0.1f}%'),
        ('Citibike Station Count', '@station_count'),
    ]
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.xgrid.visible = False
    p.ygrid.visible = False
    p.title.text_font_size = '16pt'
    p.title.align = 'center'
    
    return p

In [28]:
map_no_vehicle = create_census_map()
show(map_no_vehicle)



            geoid   name  households  \
0    86000US10001  10001     15097.0   
1    86000US10002  10002     35771.0   
2    86000US10003  10003     25080.0   
3    86000US10004  10004      1775.0   
4    86000US10005  10005      5156.0   
..            ...    ...         ...   
191  86000US11691  11691     22382.0   
192  86000US11692  11692      7981.0   
193  86000US11693  11693      5405.0   
194  86000US11694  11694      8046.0   
195  86000US11697  11697      1668.0   

                                              geometry  no_vehicle  \
0    MULTIPOLYGON (((-8238766.757 4975921.508, -823...     12699.0   
1    MULTIPOLYGON (((-8237364.465 4970257.948, -823...     29144.0   
2    MULTIPOLYGON (((-8237571.742 4972792.277, -823...     20239.0   
3    MULTIPOLYGON (((-8239611.227 4968758.136, -823...      1427.0   
4    MULTIPOLYGON (((-8239034.703 4969185.603, -823...      4488.0   
..                                                 ...         ...   
191  MULTIPOLYGON (((-8214183

“Bioinformatics and Other Bits - Choropleth Maps with Geopandas, Bokeh and Panel.” Github.io, 15 Nov. 2019, dmnfarrell.github.io/bioinformatics/bokeh-maps. Accessed 14 Dec. 2025.Contributors, Bokeh. 

“Bokeh Documentation.” Docs.bokeh.org, docs.bokeh.org/en/latest/.